## Session #3 Homework

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

Download data

In [2]:
# !curl https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv -o AB_NYC_2019.csv

Import data

In [3]:
df = pd.read_csv('data/AB_NYC_2019.csv')
print(df.shape)

(48895, 16)


In [4]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [5]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,clean_&_quiet_apt_home_by_the_park,2787,john,brooklyn,kensington,40.64749,-73.97237,private_room,149,1,9,2018-10-19,0.21,6,365
1,2595,skylit_midtown_castle,2845,jennifer,manhattan,midtown,40.75362,-73.98377,entire_home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,the_village_of_harlem....new_york_!,4632,elisabeth,manhattan,harlem,40.80902,-73.94190,private_room,150,3,0,NaN,NaN,1,365
3,3831,cozy_entire_floor_of_brownstone,4869,lisaroxanne,brooklyn,clinton_hill,40.68514,-73.95976,entire_home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,entire_apt:_spacious_studio/loft_by_central_park,7192,laura,manhattan,east_harlem,40.79851,-73.94399,entire_home/apt,80,10,9,2018-11-19,0.10,1,0


In [6]:
# subset the data
target_feature = ['price']
numerical = ['latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
categorical = ['neighbourhood_group', 'room_type']
features = categorical + numerical + target_feature
df = df[features]
df = df.fillna(0)
print(df.shape)

(48895, 10)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood_group             48895 non-null  object 
 1   room_type                       48895 non-null  object 
 2   latitude                        48895 non-null  float64
 3   longitude                       48895 non-null  float64
 4   minimum_nights                  48895 non-null  int64  
 5   number_of_reviews               48895 non-null  int64  
 6   reviews_per_month               48895 non-null  float64
 7   calculated_host_listings_count  48895 non-null  int64  
 8   availability_365                48895 non-null  int64  
 9   price                           48895 non-null  int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 3.7+ MB


---
#### Quesion 1

What is the most frequent observation (mode) for the column `'neighbourhood_group'`?

In [8]:
df.neighbourhood_group.mode()[0]

'manhattan'

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
## split 1: 80% train, 20% test
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
## split 2: from 80% test full take 25% validation (for a total of 20% from the entire dataset)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

len(df_train), len(df_val), len(df_test)

(29337, 9779, 9779)

In [11]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [12]:
y_train = df_train.price.values
y_val = df_val.price.values
y_test = df_test.price.values

del df_train['price']
del df_val['price']
del df_test['price']

---
#### Quesion 2

- Create the correlation matrix for the numerical features of your train dataset.
    - In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
- What are the two features that have the biggest correlation in this dataset?

In [13]:
df_train[numerical].corr().unstack().sort_values(ascending=False).drop_duplicates().head()

latitude           latitude                          1.000000
number_of_reviews  reviews_per_month                 0.590374
availability_365   calculated_host_listings_count    0.225913
                   number_of_reviews                 0.174477
                   reviews_per_month                 0.165376
dtype: float64

#### Make price binary
- We need to turn the price variable from numeric into binary.
- Let's create a variable `above_average` which is `1` if the price is above (or equal to) `152`.

In [14]:
## make price binary
y_train_binary = (y_train > 152).astype(int)
y_val_binary = (y_val > 152).astype(int)
y_test_binary = (y_test > 152).astype(int)

---
#### Quesion 3

- Calculate the mutual information score with the (binarized) price for the two categorical variables that we have. Use the training set only.
- Which of these two variables has bigger score?
- Round it to 2 decimal digits using `round(score, 2)`

In [15]:
from sklearn.metrics import mutual_info_score

In [16]:
score = mutual_info_score(df_train.neighbourhood_group, y_train_binary)
print(round(score,2))

0.05


In [17]:
score = mutual_info_score(df_train.room_type, y_train_binary)
print(round(score, 2))

0.14


_Room type has a higher score_

---
#### Quesion 4

- Now let's train a logistic regression
- Remember that we have two categorical variables in the data. Include them using one-hot encoding.
- Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)`
- Calculate the accuracy on the validation dataset and rount it to 2 decimal digits.

In [18]:
from sklearn.feature_extraction import DictVectorizer

In [19]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.fit_transform(val_dict)

In [20]:
from sklearn.linear_model import LogisticRegression

In [21]:
model = LogisticRegression(solver='lbfgs', C=1.0, max_iter=10000, random_state=42)
model.fit(X_train, y_train_binary)

LogisticRegression(max_iter=10000, random_state=42)

In [22]:
model.intercept_[0]

-0.07404284655942911

In [23]:
model.coef_[0]

array([ 3.05329551e-03,  3.50401214e-03, -5.83938728e+00, -3.17797222e+00,
       -1.17047951e-02, -9.94979736e-02,  1.45447669e-01,  1.59652676e+00,
       -5.72227690e-03, -1.70479442e+00, -3.29226785e-03, -4.21277395e-02,
        1.95768434e+00, -8.22726511e-01, -1.20299807e+00])

In [24]:
y_pred = model.predict_proba(X_val)[:, 1]

In [25]:
above_average_pred = (y_pred >= 0.5)

In [26]:
(y_val_binary == above_average_pred).mean()

0.7903671132017589

In [27]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = above_average_pred.astype(int)
df_pred['actual'] = y_val_binary

In [28]:
df_pred['correct'] = df_pred.prediction == df_pred.actual

In [29]:
df_pred.correct.mean()

0.7903671132017589

In [30]:
df_pred.head()

,probability,prediction,actual,correct
0,0.028650,0,0,True
1,0.595998,1,0,False
2,0.425355,0,1,False
3,0.074522,0,0,True
4,0.810811,1,1,True


---
### Question 5
- We have 9 features: 7 numerical features and 2 categorical.
- Let's find the least useful one using the feature elimination technique.
- Train a model with all these features (using the same parameters as in Q4).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
- Which of following feature has the smallest difference?
    - neighbourhood_group
    - room_type
    - number_of_reviews
    - reviews_per_month

> note: the difference doesn't have to be positive

In [31]:
model.score(X_train, y_train_binary)

0.7933326516003681

In [32]:
def logistic_regression(df_train, df_val, y_train_binary=y_train_binary, y_val_binary=y_val_binary):
    dv = DictVectorizer(sparse=False)

    train_dict = df_train.to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    val_dict = df_val.to_dict(orient='records')
    X_val = dv.fit_transform(val_dict)

    model = LogisticRegression(solver='lbfgs', C=1.0, max_iter=10000, random_state=42)
    model.fit(X_train, y_train_binary)

    y_pred = model.predict_proba(X_val)[:, 1]
    above_average_pred = (y_pred >= 0.5)
    score = (y_val_binary == above_average_pred).mean()
    
    return score

In [33]:
def remove_feature(df, feature):
    return df.drop(feature, axis=1)

In [34]:
features = []
diff = []

# baseline model
score_all = logistic_regression(df_train, df_val)

for feature in df_train.columns:
    print(feature)
    df_train_temp = remove_feature(df_train, feature)
    df_val_temp = remove_feature(df_val, feature)
    score = logistic_regression(df_train_temp, df_val_temp)
    diff.append(score_all - score)
    features.append(feature)

neighbourhood_group
room_type
latitude
longitude
minimum_nights
number_of_reviews
reviews_per_month
calculated_host_listings_count
availability_365


In [39]:
diff_dict = {k:v for k,v in zip(features, diff)}
diff_df = pd.DataFrame.from_dict(diff_dict, orient='index', columns=['diff'])

diff_df.apply(np.abs).sort_values('diff')

,diff
reviews_per_month,0.000102
minimum_nights,0.000818
number_of_reviews,0.001023
calculated_host_listings_count,0.001432
longitude,0.004090
latitude,0.004499
availability_365,0.009715
neighbourhood_group,0.040290
room_type,0.060742


Removing the feature `reviews_per_month` has the least effect on the model accuracy

---

### Question 6
- For this question, we'll see how to use a linear regression model from Scikit-Learn
- We'll need to use the original column `'price'`. Apply the logarithmic transformation to this column.
- Fit the Ridge regression model on the training data.
- This model has a parameter `alpha`. Let's try the following values: `[0, 0.01, 0.1, 1, 10]`
- Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.

If there are multiple options, select the smallest `alpha`.

In [36]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [37]:
scores = []
alphas = []
for alpha in [0, 0.01, 0.1, 1, 10]:
    print(alpha)
    model = Ridge(alpha=alpha)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    scores.append(rmse)
    alphas.append(alpha)

0
0.01
0.1
1
10


In [38]:
scores_dict = {k:v for k,v in zip(alphas, scores)}
scores_df = pd.DataFrame.from_dict(scores_dict, orient='index', columns=['score'])

np.round(scores_df.sort_values('score'),3)

,score
0.10,211.903
0.01,211.903
1.00,211.903
0.00,211.928
10.00,211.950


The `alpha` values with the smallest rmse are `0.1`, `0.01` and `1.0`